In [43]:
import collections

class Body_object:
    
    def __init__(self, line):
        self.line = line
        if("\n" not in line):
            self.line = line + "\n"
        self.chrom = ""
        self.id = ""
        self.ref = ""
        self.alt = ""
        self.qual = ""
        self.filter = ""
        self.info = ""
        self.extract_data_from_line()
        
    def extract_data_from_line(self):
        splitted_line = self.line.split("\t")
        self.chrom = splitted_line[0]
        self.id = splitted_line[1]
        self.ref = splitted_line[2]
        self.alt = splitted_line[3]
        self.qual = splitted_line[4]
        self.filter = splitted_line[5]
        self.info = splitted_line[6]
        
    #potrebno mi je da vidim kakve su zakonitosti pri spajanju
    def __eq__(self, other):   
        return self.line == other.line

class Generic_header_object:
    def __init__(self, line, tag):
        self.line = line
        self.tag = tag
        self.data = {}
        self.extract_line_data()
                
    def extract_line_data(self):
        
        try:
            line_parse = self.line[len(self.tag)+3 : -1]
            attributes = line_parse.split(',')
            for item in attributes:
                if item.count('=') >= 1:
                    splitted = item.split('=', 1)
                    self.data[splitted[0].lstrip('<,"')] = splitted[1].rstrip('>')
        except:
             pass
        
        
    def __eq__(self, other):   
        return self.line == other.line

class Input_file:
    current_tag = ""
    
    def __init__(self, path):
        self.path = path
        self.has_current_tag = False
        self.header_read = False
        self.eof = False
    
    def set_file(self, file):
        self.file = file
    
    def set_next_tag(self, next_tag):
        self.next_tag = next_tag
        
    def set_value_has_current_tag(self):
        self.has_current_tag = (self.current_tag == self.next_tag)
        

def read_specific_tag_header_in_files(list_of_files):
    
    for item in list_of_files:
        if(item.has_current_tag):
            previous_position_of_file = item.file.tell()
            line_of_file = item.file.readline()
            #print(line_of_file)
        
            while(line_of_file.startswith(Input_file.current_tag)):
                create_generic_header_object_add_to_list(Input_file.current_tag[2:], line_of_file)
                previous_position_of_file = item.file.tell()
                line_of_file = item.file.readline()
            
            item.file.seek(previous_position_of_file)
    
    write_tag_in_file()      

def read_specific_chrom_in_files(list_of_files):
    
    for item in list_of_files:
        if(item.has_current_tag):
            previous_position_of_file = item.file.tell()
            line_of_file = item.file.readline()
        
            while(line_of_file.startswith(Input_file.current_tag)):
                create_body_object_add_to_list(line_of_file)
                previous_position_of_file = item.file.tell()
                line_of_file = item.file.readline()
            
            #rewind file 
            item.file.seek(previous_position_of_file)
    
    write_tag_in_file() 
    
    
def get_next_tag(list_of_files):
    
    tags = []
    
    for item in list_of_files:
        previous_position_of_file = item.file.tell()
        
        next_line = item.file.readline()
        if(next_line.split("=")[0].startswith("##")):
            item.set_next_tag(next_line.split("=")[0]) 
            
            tags.append(item.next_tag)  
        else:
            item.set_next_tag("") 
        item.file.seek(previous_position_of_file)
        
    Input_file.current_tag = (list(sorted(tags)))[0] if len(tags) > 0 else ""

    for item in list_of_files:
        item.set_value_has_current_tag()
        
        
def write_tag_in_file():
    
    list_of_lines.sort(key = lambda x : x.line)
    for value in list_of_lines:
        merged_vcf_file.write(value.line)    
    
    list_of_lines.clear() 

def create_generic_header_object_add_to_list(tag, line):
    generic_header_object = Generic_header_object(line, tag)   
    if generic_header_object not in list_of_lines:
         list_of_lines.append(generic_header_object)
            
def create_body_object_add_to_list(line):
    body_object = Body_object(line)
    if body_object not in list_of_lines:
        list_of_lines.append(body_object)
            
    
def get_next_chromosome(list_of_files):
    chroms = []
    
    for item in list_of_files:
        previous_position_of_file = item.file.tell()
        
        next_line = item.file.readline()
        if(next_line == ""):
            item.eof = True
        else:
            item.set_next_tag((next_line.split("\t"))[0])
            chroms.append(item.next_tag)
            item.file.seek(previous_position_of_file)
    
    if(len(chroms) > 0):
        Input_file.current_tag = (list(sorted(chroms)))[0] if len(chroms) > 0 else ""
    
        #set files which has current tag
        for item in list_of_files:
            item.set_value_has_current_tag()
        
def verify_start_of_header(list_of_files):
    for item in list_of_files:
        previous_position_of_file = item.file.tell()
        
        next_line = item.file.readline()
        if(next_line.startswith("#CHROM")):
            pass
        else:
            return False
    
    merged_vcf_file.write(next_line)
    return True
        



first_file_path = "../Fajlovi/1000G_test.vcf"
second_file_path = "../Fajlovi/1000G_test_2.vcf"

merged_vcf_file = open("merged.vcf","w+")             
        
first_file = Input_file(first_file_path) 
second_file = Input_file(second_file_path)

list_of_files = [first_file, second_file]

list_of_lines = list()

not_end_of_header = True
not_end_of_body = True

with open(list_of_files[0].path) as list_of_files[0].file:
    with open(list_of_files[1].path) as list_of_files[1].file:
        
        while not_end_of_header:
            get_next_tag(list_of_files)
            if Input_file.current_tag.startswith("##"):
                read_specific_tag_header_in_files(list_of_files)
            else:
                not_end_of_header = False
        
        if(verify_start_of_header(list_of_files)):

            while not_end_of_body:
                get_next_chromosome(list_of_files)
                if(sum(1 for item in list_of_files if item.eof == False) > 0):
                    if(Input_file.current_tag != ""):
                        read_specific_chrom_in_files(list_of_files)
                else:
                    not_end_of_body = False     
    